In [1]:
"""
General Data Science Packages
"""
import numpy as np
import pandas as pd
import geopandas as gpd
# import fiona
# import shapely
# from shapely.geometry import shape

"""
Data Managment Packages
"""
# import time
# import os
import ast

"""
Geocoding Packages
"""
# import geopy as gp
# from geopy.geocoders import Nominatim
# from geopy.extra.rate_limiter import RateLimiter
# from functools import partial

"""
Distance Calculations
"""
# from geopy.distance import geodesic
# from geopy.distance import great_circleQ

"""
Check Python Version
"""
!python --version 

Python 3.8.3


# 1. Generate Multilocations

## 1.0 Prepare the Raw Data

In [2]:
# load & preview the csv file
survey_ = pd.read_csv('data_raw/raw_survey/210615_raw_survey.csv')

print('\nRows, Columns: {}'.format(survey_.shape))
survey_.head(3)


Rows, Columns: (1644, 332)


,SITE_CODE,TARGET_LATITUDE,TARGET_LONGITUDE,SITE_LATITUDE,SITE_LONGITUDE,SURVEY DURATION IN MINUTES,DATE_UPLOADED,3 Migrant,4 Gender,4 Other Gender,...,236 Current Residence,Migrant Quota Category,237 Returned,238 Places Returned To,Unnamed: 326,Unnamed: 327,Unnamed: 328,Unnamed: 329,Unnamed: 330,Unnamed: 331
0,276785,-1.21003,36.78920,-1.28892,36.80449,139.96,3/4/21,Yes,Male,-1,...,More than 2 years,Medium-Term Resident,No,-1,NaN,NaN,NaN,NaN,NaN,NaN
1,276788,-1.21010,36.78923,-1.28872,36.80430,74.59,3/4/21,Yes,Male,-1,...,-1,NaN,-1,-1,NaN,NaN,NaN,NaN,NaN,NaN
2,276802,-1.21010,36.78923,-1.28884,36.80433,244.90,3/5/21,Yes,Male,-1,...,One year exactly or/More than a year,New Resident,No,-1,NaN,NaN,NaN,NaN,NaN,NaN


In [3]:
# run to reset the working dataframe to original dataframe
survey = survey_.copy()

### 1.0.a. Prepare the index of the input data

##### Set the index to 'SITE_CODE'

To ensure the indices of any exported datasets remain consistent with the original raw data and has the capability to be appended back, whenever applicable, the respondent's survey ID will be used as the index.

In [4]:
# check for any potential row-wise errors
print('\nDuplicate rows?:')
print(survey.duplicated().value_counts())

print('\nUnique SITE_CODE column?:')
print(survey['SITE_CODE'].is_unique)

# duplicate the respondent ID column and set it as the index
survey['ID'] = survey['SITE_CODE']
survey = survey.set_index('ID')

# check if the uniqueness was maintained
print('\nUnique ID index?:')
print(survey.index.is_unique)

survey.head(3)


Duplicate rows?:
False    1644
dtype: int64

Unique SITE_CODE column?:
True

Unique ID index?:
True


,SITE_CODE,TARGET_LATITUDE,TARGET_LONGITUDE,SITE_LATITUDE,SITE_LONGITUDE,SURVEY DURATION IN MINUTES,DATE_UPLOADED,3 Migrant,4 Gender,4 Other Gender,...,236 Current Residence,Migrant Quota Category,237 Returned,238 Places Returned To,Unnamed: 326,Unnamed: 327,Unnamed: 328,Unnamed: 329,Unnamed: 330,Unnamed: 331
ID,,,,,,,,,,,,,,,,,,,,,
276785,276785,-1.21003,36.78920,-1.28892,36.80449,139.96,3/4/21,Yes,Male,-1,...,More than 2 years,Medium-Term Resident,No,-1,NaN,NaN,NaN,NaN,NaN,NaN
276788,276788,-1.21010,36.78923,-1.28872,36.80430,74.59,3/4/21,Yes,Male,-1,...,-1,NaN,-1,-1,NaN,NaN,NaN,NaN,NaN,NaN
276802,276802,-1.21010,36.78923,-1.28884,36.80433,244.90,3/5/21,Yes,Male,-1,...,One year exactly or/More than a year,New Resident,No,-1,NaN,NaN,NaN,NaN,NaN,NaN


### 1.0.b. Prepare the columns of the input data

##### Add country columns 8b and 228b corresponding to city columns 8 and 228

To generate the multilocations for some of the columns, it will be important to have the corresponding country names for the city columns _'8 Identify City'_ and _'228 Identify the City'_.
\
\
Although corresponding country columns do not exist within the raw data, with the small number of cities in this column, we can add the country values by mapping over these columns with a conditional.
* Johannesburg, __South Africa__
* Accra, __Ghana__
* Nairobi, __Kenya__ 

In [5]:
def add_country(city): 
    """
    A function to return a string of the country based on the city response for 
    '8 Identify City'
    
    RETURNS: A string, either 'South Africa', 'Ghana', or 'Kenya'. If the city is null or '-1'
    returns '-1'. If the city is none of these values, returns '0' for error-checking.
    """
    
    if (city == 'Johannesburg') | (city == 'Johannesburg '): return 'South Africa'
    elif (city == 'Accra') | (city == 'Accra '): return 'Ghana'
    elif (city == 'Nairobi') | (city == 'Nairobi '): return 'Kenya'
    elif (city == '-1'): return '-1'
    else: return '0'

# vectorized function 
add_country = np.vectorize(add_country)

In [6]:
# Add new country columns for later
survey['8b Identify Country'] = add_country( survey['8 Identify City '] )
survey['228b Identify the Country'] = add_country( survey['228 Identify the City'] )

# Diagnostics
print('\n8  &  8b')
print('------------')
print(survey['8 Identify City '].value_counts(dropna=False))
print(survey['8b Identify Country'].value_counts(dropna=False))

print('\n228  &  228b')
print('------------')
print(survey['228 Identify the City'].value_counts(dropna=False))
print(survey['228b Identify the Country'].value_counts(dropna=False))

print('\nMatches:')
print((survey['8b Identify Country'] == survey['228b Identify the Country']).value_counts())


8  &  8b
------------
Accra           581
Nairobi         548
Johannesburg    515
Name: 8 Identify City , dtype: int64
Ghana           581
Kenya           548
South Africa    515
Name: 8b Identify Country, dtype: int64

228  &  228b
------------
Nairobi         483
Accra           443
Johannesburg    368
-1              350
Name: 228 Identify the City, dtype: int64
Kenya           483
Ghana           443
South Africa    368
-1              350
Name: 228b Identify the Country, dtype: int64

Matches:
True     1293
False     351
dtype: int64


##### Columns and Column Positions within the Data

For reference while in the process of running through the notebook. \
\
Note that some column names have whitespaces at the end. To simplify any future process of appending exported data back without "middleman" files, these were left as is.

In [7]:
# print columns and their position in the survey
for i, col in enumerate(list(survey.columns)):
    print(str(i) + '\t' + str(col))

0	SITE_CODE
1	TARGET_LATITUDE
2	TARGET_LONGITUDE
3	SITE_LATITUDE
4	SITE_LONGITUDE
5	SURVEY DURATION IN MINUTES
6	DATE_UPLOADED
7	3 Migrant
8	4 Gender
9	4 Other Gender
10	5 Quota Check 
11	7 Date
12	8 Identify City 
13	9 Johannesburg Neighbourhoods
14	9 Accra Neighbourhoods 
15	9 Nairobi Neighbourhoods
16	10 Housing Type
17	10 Other Housing
18	100: Village, City or Town
19	100: Village, City or Town Name
20	100b: Neighbourhood
21	101 Nearest Town
22	102 Time to City by Bus
23	103 Country of Origin
24	104 Year of Birth
25	106 Ethnic Group
26	106 Language
27	107 Marital Status
28	108 Number of Partners
29	109 Partner Location
30	111 Education 
31	111 Other
32	112 Additional Training
33	113 Additional Training
34	113 Other Additional Training
35	115 Mother Tongue
36	116 Languages
37	121 Learnt Languages 
38	121 Which Languages
39	123 Languages in Neighbourhood
40	123 Which Languages
41	125 Children
42	125 Number of Children 
43	126 Children by Birth
44	127 Born Since Leaving
45	132 People 

## 1.1 Reformat the Location Columns into Multilocations

##### Structure of the Multilocations

To reduce ambiguity and misinterpretations of the locations names and maintain a consistent structure for how they are stored in the dataframe, all columns denoting locations in the survey will be reformatted into "multilocations" for the gazetteer, constructed from related location names.

The general structure will include three values as follows:

__L1__ &emsp; Precise location \
__L2__ &emsp; Reference location \
__L3__ &emsp; Country 

##### Special Note on L2 of Multilocations

Note that __L2__ is a _reference point_ intended to be used to help disambiguage the precise location. The relationship between the precise location __L1__ and reference location __L2__ will NOT always be consistent semantically. 

For most of the columns (see 1.1.a.), the response is structured so that __L2__ will indicate the _nearest city_ .

__L1__ &emsp; Village/Town/City or  neighbourhood \
__L2__ &emsp; Nearest City for village or town \
__L3__ &emsp; Country

In some responses for __L2__, nearest city values were actually fairly distant from the precise location or indicated another location than the nearest city. These will be maintained "as is", but for this reason, it is important to keep in mind that __L2__ may not always be helpful towards disambiguating the precise location.

In column where __L1__ indicate neighborhoods rather than cities, no response to suggest the nearest city is available. For 100 series questions pertaining the origin neighborhood, the corresponding value for origin city is used for __L2__. If it one of the 200 or 300 series questions specific to arrival at the interview city, the _interview city_ is used instead (see 1.1.b). 

For any remaining columns where neither of these value are available, the same value for __L1__ is duplicated into __L2__. 

Regardless of their semantic relationship, the lat-lon coordinates returned by Nominatim for __L1__ and __L2__ should be related by proximity. As the places are collected and represented as _points_ and not _polygons_ , the relationship between __L1__ and __L2__ will be tied together by relatively close distances between the two points, independent of whether or not the boundaries are coterminous.


##### Array and Tuple Structure

Although these will initially be stored as numpy arrays for performance with several of the Numpy and Pandas functions used, before they are exported to a csv, they will be converted into base Python tuples for wider compatibility and readibility. 

__[  L1  ,  L2  ,  L3  ]__\
__(  L1  ,  L2  ,  L3  )__

##### Accomodating multiple multilocations

In several cases, repondents entered multiple locations for a single question. To accomodate the responses with multiple location entries while continuing to maintain a consistent structure across the gazetteer, responses will be stored as multidimensional arrays or an "array within an array" regardless of whether the reponse contains single or multiple locations.

_Single Location Entry:_ \
__[ [ L1 ,  L2  ,  L3 ] ]__\
__( ( L1 ,  L2  ,  L3 ) )__\
\
_Multiple Location Entries:_ \
__[ [ L1 ,  L2  ,  L3 ], [ L1 ,  L2  ,  L3 ], ...]__\
__( ( L1 ,  L2  ,  L3 ), ( L1 ,  L2  ,  L3 ), ...)__

### 1.1.a. Generate Multilocations by Splitting by the Delimiters

The majority responses within the raw data are are given as a string with the following delimiter characters: \
\
__~__ &emsp; _Separates discrete entries._ \
__^__ &emsp; _Separates components of a discrete entry._ \

Three location names are given, which indicate:
* Village/Town/City or  neighbourhood
* Nearest City for village or town 
* Country

For these columns, the string is split by these delimiter characters into a temporary list. The location names will be called by their positional index, which should remain consistent through the column, then be placed into the multilocation array  within the temporary list.

##### Keys for the splits

In [8]:
# dictionary of keys for the splits
d_splits = {
    
    '109': ['109 Partner Location', 1, 2, 3],
    '127': ['127 Born Since Leaving', 1, 2, 3],
    '134': ['134 Adults in Household', 1, 2, 3],
    '142': ['142 Households Members in Different Place', 1, 1, 2],
    '152': ['152 Frequency of Visits', 1, 1, 2],
    '201': ['201 Place and Duration', 0, 1, 2],
    '202': ['202 Departure and Duration', 0, 1, 2],
    '206': ['206 Migration ', 0, 1, 2],
    '211': ['211 Migrated where', 0, 1, 2],
    '214b': ['214b Migration', 0, 1, 2],
    '215': ['215 Refugee Camps', 0, 1, 2],
    '219': ['219 Aspiration', 0, 1, 2],
    '225': ['225 Location of Relations', 1, 2, 3],
    '311': ['311 Frequency and Amount', 1, 2, 3],
    '321': ['321 Frequency and Amount', 1, 2, 3],
    '340': ['340 Investment Elsewhere', 0, 1, 2],
    '341': ['341 Obligations Where', 0, 1, 2],
    '342': ['342 Disappointed', 1, 2, 3],
    '343': ['343 Location of Properties ', 0, 1, 2],
    '344': ['344 Somewhere else', 0, 1, 2],
    '345': ['345 Self Sacrifice', 0, 1, 2],
    '406': ['406 Who Did You Contact', 1, 2, 3],
    '516': ['516 News From Where', 0, 1, 2],
    '701': ['701 Where to', 0, 1, 2],
    '703': ['703 Where to ', 0, 1, 2],
    '705': ['705 Where to', 0, 1, 2],
    '707': ['707 Where to', 0, 1, 2],
    '712': ['712 Specify Where', 0, 1, 2],
    '713': ['713 Specify where', 0, 1, 2],
    '715': ['715 Specify where', 0, 1, 2],
    '717': ['717 Specify Where', 0, 1, 2]
    
}

# dataframe of keys for the splits
df_splits = pd.DataFrame(d_splits, index=['c', 'i1', 'i2', 'i3'])
df_splits

,109,127,134,142,152,201,202,206,211,214b,...,406,516,701,703,705,707,712,713,715,717
c,109 Partner Location,127 Born Since Leaving,134 Adults in Household,142 Households Members in Different Place,152 Frequency of Visits,201 Place and Duration,202 Departure and Duration,206 Migration,211 Migrated where,214b Migration,...,406 Who Did You Contact,516 News From Where,701 Where to,703 Where to,705 Where to,707 Where to,712 Specify Where,713 Specify where,715 Specify where,717 Specify Where
i1,1,1,1,1,1,0,0,0,0,0,...,1,0,0,0,0,0,0,0,0,0
i2,2,2,2,1,1,1,1,1,1,1,...,2,1,1,1,1,1,1,1,1,1
i3,3,3,3,2,2,2,2,2,2,2,...,3,2,2,2,2,2,2,2,2,2


##### Function to split

In [9]:
# function to split
def delimit(string):
    """
    A function that splits a string by its delimiters
    
    RETURNS: Separated strings in an array
    """
    # 01 | split the string into discrete substring entries using delimiter '~'
    sub_s = str(string).split('~')
    
    # 02 | split each substring entry into subarrays with discrete component using delimiter '^'
    a = [str(item).split('^') for item in sub_s]
    
    return a

# function to split and select
def ml_split(string, i1, i2, i3):   
    """
    A function that splits a string by its delimiters and selects values 
    to place into the multilocation array based on positional indices passed in.
    
    RETURNS: An array of multilocation array(s).
    """
    a = delimit(string)
    
    # 03 | generate a new array by positionally pulling the appropriate components
    ml = [[str(sub[i1]).strip(),
           str(sub[i2]).strip(),
           str(sub[i3]).strip()] for sub in a if (len(sub) >= i3+1)]
    # * the conditional prevents the function from pulling responses that are completely null
    
    return ml

##### Loop to perform the splits

In [10]:
# 01 | Create a new dataframe to store the multilocations
mls_A = pd.DataFrame(index   = survey.index,
                     columns = df_splits.columns)

# 02 | For each split to be performed...
for newcol in list(df_splits.columns):
    
    # 02-a | Get each of the keys from the splits surveyframe
    c  = str( df_splits.loc['c',  newcol] )
    i1 = int( df_splits.loc['i1', newcol] )
    i2 = int( df_splits.loc['i2', newcol] )
    i3 = int( df_splits.loc['i3', newcol] )
    
    # 02-b | Reset the column and its dtype
    mls_A.at[:, newcol] = None
    
    # 02-c | Map over the rows of the survey with the function
    mls_A[newcol] = survey.apply(lambda row : ml_split(row[c], i1, i2, i3), axis = 1)

# Diagnostics
print('\nRows, Columns: {}'.format(mls_A.shape))
print(pd.Series(mls_A.index == survey.index).value_counts())

# Preview
mls_A.head(10)


Rows, Columns: (1644, 31)
True    1644
dtype: int64


,109,127,134,142,152,201,202,206,211,214b,...,406,516,701,703,705,707,712,713,715,717
ID,,,,,,,,,,,,,,,,,,,,,
276785,[],[],"[[Kawango, Kisumu, Kenya], [Kawango, Kisumu, K...",[],[],"[[Kawango, Kisumu, Kenya]]","[[Manyatta, Kisumu, Kenya], [Kawangware, Nairo...",[],[],[],...,[],[],[],[],[],[],[],[],[],[]
276788,[],"[[N/A, N/A, N/A]]","[[N/A, N/A, N/A]]",[],[],[],[],[],[],[],...,[],[],[],[],[],[],[],[],[],[]
276802,[],[],"[[Awendo, Migori, Kenya]]",[],[],"[[Machakos, Machakos, Kenya], [Utawala, Nairob...",[],[],[],[],...,"[[Oyugis, Homabay, Kenya]]",[],"[[Utawala, Nairobi, Kenya]]","[[Soweto, Pretoria, South Africa]]","[[Enugu, Abuja, Nigeria]]",[],"[[Utawala, Nairobi, Kenya]]","[[Utawala, Nairobi, Kenya]]","[[Oyugis, Homabay, Kenya]]","[[Jomvu, Mombasa, Kenya]]"
276814,[],"[[Nairobi, Nairobi, Kenya], [London, Nakuru, K...","[[Bondo, Siaya, Kenya]]","[[Kisumu, Kisumu, Kenya]]",[],"[[Wathorego, Kisumu, Kenya]]","[[London, Nakuru, Kenya]]","[[Lolgoria, Narok, Kenya]]","[[Londiani, Kericho, Kenya]]",[],...,[],[],[],[],[],[],[],[],[],[]
276822,[],[],"[[None, None, None]]",[],[],"[[Kawango, Maseno, Kenya]]","[[Kedowa, Kericho, Kenya]]",[],[],[],...,"[[Kawango, Kisumu, Kenya]]",[],[],[],[],[],[],[],[],[]
276931,[],"[[London, Nakuru, Kenya]]","[[Usenge, Siaya, Kenya]]","[[Mombasa, Mombasa, Kenya]]","[[Mombasa, Mombasa, Kenya]]","[[Mamboleo, Kisumu, Kenya]]",[],[],[],[],...,[],[],[],[],[],[],[],[],[],[]
276932,[],"[[Kawangware, Nairobi, Kenya]]","[[Na, Na, Na]]",[],[],"[[Kajulu, Kisumu, Kenya]]",[],[],[],[],...,[],"[[Na, Na, Na]]",[],[],[],[],[],[],[],[]
276934,[],[],"[[Nairobi, Nairobi, Kenya]]",[],[],"[[Pumwani, Nairobi, Kenya]]","[[Kericho, Kericho, Kenya], [Kawangware, Nairo...",[],[],[],...,"[[Nairobi, Nairobi, Kenya]]",[],[],[],[],[],[],[],[],[]
276935,[],"[[Nairobi, Nairobi, Kenya]]","[[Wote, Wote, Kenya]]",[],[],"[[Wote, Wote, Kenya]]","[[Kawangware, Nairobi, Kenya]]",[],[],[],...,"[[Wote, Wote, Kenya]]","[[Makueni, Makueni, Kenya]]",[],[],[],[],[],[],[],[]


### 1.1.b. Generate Multilocations by Binding Related Columns Together

Some location responses in the survey are given as singular strings without additional components. Almost all of these have implied geographic relationships with the responses other questions, such as questions that ask for the current neighborhood if home and work within the interview city. 

To preserve these locations, multilocations are alternatively generated by combining these individual strings row-wise.

##### Keys for the binds

In [11]:
# dictionary of keys for the binds
d_binds = {
    
    '9J': ['9 Johannesburg Neighbourhoods', '228 Identify the City', '228b Identify the Country'],
    '9A': ['9 Accra Neighbourhoods ', '228 Identify the City', '228b Identify the Country'],
    '9N': ['9 Nairobi Neighbourhoods', '228 Identify the City', '228b Identify the Country'],
    '100b': ['100b: Neighbourhood', '100: Village, City or Town Name', '103 Country of Origin'],
    '101': ['101 Nearest Town', '101 Nearest Town', '103 Country of Origin'],
    '302': ['302 Original Place of Work', '228 Identify the City', '228b Identify the Country'],
    '308': ['308 Work Location', '228 Identify the City', '228b Identify the Country'],
    '229w': ['229 First Week', '228 Identify the City', '228b Identify the Country'],
    '229x': ['229 First Week9', '228 Identify the City', '228b Identify the Country'],
    '229y': ['229 First Week10', '228 Identify the City', '228b Identify the Country'],
    '229z': ['229 Specify other ', '228 Identify the City', '228b Identify the Country']
    
}

# dataframe of keys for the binds
df_binds = pd.DataFrame(d_binds, index=['c1', 'c2', 'c3'])
df_binds

,9J,9A,9N,100b,101,302,308,229w,229x,229y,229z
c1,9 Johannesburg Neighbourhoods,9 Accra Neighbourhoods,9 Nairobi Neighbourhoods,100b: Neighbourhood,101 Nearest Town,302 Original Place of Work,308 Work Location,229 First Week,229 First Week9,229 First Week10,229 Specify other
c2,228 Identify the City,228 Identify the City,228 Identify the City,"100: Village, City or Town Name",101 Nearest Town,228 Identify the City,228 Identify the City,228 Identify the City,228 Identify the City,228 Identify the City,228 Identify the City
c3,228b Identify the Country,228b Identify the Country,228b Identify the Country,103 Country of Origin,103 Country of Origin,228b Identify the Country,228b Identify the Country,228b Identify the Country,228b Identify the Country,228b Identify the Country,228b Identify the Country


##### Function to bind

In [14]:
# function to perform the binds
def ml_bind(l1, l2, l3):
    """
    A function that binds column values into a multilocation array
    based on the column names passed in.
    
    RETURNS: An array of multilocation array(s).
    """
    # 01 | Create a subarray of strings from the values passed in 
    sub = np.array([str(l1).strip(),
                      str(l2).strip(),
                      str(l3).strip()], dtype='str')
    
    # 02 | Place the array into an array to match structure
    ml = np.array([sub])
    
    return ml

##### Loop to perform the binds

In [15]:
# 01 | Create a new dataframe to store the multilocations
mls_B = pd.DataFrame(index   = survey.index,
                     columns = df_binds.columns)


# 02 | For each bind to be performed...
for newcol in list(df_binds.columns):
    
    # 02-a | Get each of the keys from the binds dataframe
    c1 = str( df_binds.loc['c1', newcol] )
    c2 = str( df_binds.loc['c2', newcol] )
    c3 = str( df_binds.loc['c3', newcol] )
    
    # 02-b | Reset the column and its datatype
    mls_B.at[:, newcol] = None
    
    # 02-c | Map over the rows of the data with the function
    mls_B[newcol] = survey.apply(lambda row : ml_bind(row[c1], row[c2], row[c3]), axis=1)

# Diagnostics
print('\nRows, Columns: {}'.format(mls_B.shape))
print('\nIndex matches original:')
print(pd.Series(mls_B.index == survey.index).value_counts())

# Preview
mls_B.head(10)


Rows, Columns: (1644, 11)

Index matches original:
True    1644
dtype: int64


,9J,9A,9N,100b,101,302,308,229w,229x,229y,229z
ID,,,,,,,,,,,
276785,"[[-1, Nairobi, Kenya]]","[[-1, Nairobi, Kenya]]","[[Kawangware, Nairobi, Kenya]]","[[-1, Kawango, Kenya]]","[[Kisumu, Kisumu, Kenya]]","[[-3, Nairobi, Kenya]]","[[-3, Nairobi, Kenya]]","[[-1, Nairobi, Kenya]]","[[Kawangware, Nairobi, Kenya]]","[[-1, Nairobi, Kenya]]","[[-1, Nairobi, Kenya]]"
276788,"[[-1, -1, -1]]","[[-1, -1, -1]]","[[Kawangware, -1, -1]]","[[-1, Kokal, Kenya]]","[[Oyugis, Oyugis, Kenya]]","[[-1, -1, -1]]","[[-1, -1, -1]]","[[-1, -1, -1]]","[[-1, -1, -1]]","[[-1, -1, -1]]","[[-1, -1, -1]]"
276802,"[[-1, Nairobi, Kenya]]","[[-1, Nairobi, Kenya]]","[[Kawangware, Nairobi, Kenya]]","[[-1, Oyugjs, Kenya]]","[[Oyugis, Oyugis, Kenya]]","[[Utawala, Nairobi, Kenya]]","[[Nairobi Cbd, Nairobi, Kenya]]","[[-1, Nairobi, Kenya]]","[[Roysambu, Nairobi, Kenya]]","[[-1, Nairobi, Kenya]]","[[-1, Nairobi, Kenya]]"
276814,"[[-1, Nairobi, Kenya]]","[[-1, Nairobi, Kenya]]","[[Kayole, Nairobi, Kenya]]","[[-1, Wathorego, Kenya]]","[[Kisumu, Kisumu, Kenya]]","[[Soweto, Nairobi, Kenya]]","[[-3, Nairobi, Kenya]]","[[-1, Nairobi, Kenya]]","[[Kayole, Nairobi, Kenya]]","[[-1, Nairobi, Kenya]]","[[-1, Nairobi, Kenya]]"
276822,"[[-1, Nairobi, Kenya]]","[[-1, Nairobi, Kenya]]","[[Kayole, Nairobi, Kenya]]","[[-3, Kisumu, Kenya]]","[[-1, -1, Kenya]]","[[Soweto, Nairobi, Kenya]]","[[Kayole, Nairobi, Kenya]]","[[-1, Nairobi, Kenya]]","[[Buru Buru, Nairobi, Kenya]]","[[-1, Nairobi, Kenya]]","[[-1, Nairobi, Kenya]]"
276931,"[[-1, Nairobi, Kenya]]","[[-1, Nairobi, Kenya]]","[[Kawangware, Nairobi, Kenya]]","[[-1, Kajulu, Kenya]]","[[Kisumu, Kisumu, Kenya]]","[[Mama Oliech, Nairobi, Kenya]]","[[Stage, Nairobi, Kenya]]","[[-1, Nairobi, Kenya]]","[[Kayole, Nairobi, Kenya]]","[[-1, Nairobi, Kenya]]","[[-1, Nairobi, Kenya]]"
276932,"[[-1, Nairobi, Kenya]]","[[-1, Nairobi, Kenya]]","[[Kawangware, Nairobi, Kenya]]","[[-1, Kajulu, Kenya]]","[[Kisumu, Kisumu, Kenya]]","[[Bus Park, Nairobi, Kenya]]","[[Kawangware, Nairobi, Kenya]]","[[-1, Nairobi, Kenya]]","[[Kawangware, Nairobi, Kenya]]","[[-1, Nairobi, Kenya]]","[[-1, Nairobi, Kenya]]"
276934,"[[-1, -1, -1]]","[[-1, -1, -1]]","[[Kawangware, -1, -1]]","[[Pumwani, Nairobi, Kenya]]","[[-1, -1, Kenya]]","[[Settilite, -1, -1]]","[[Kawangware, -1, -1]]","[[-1, -1, -1]]","[[-1, -1, -1]]","[[-1, -1, -1]]","[[-1, -1, -1]]"
276935,"[[-1, Nairobi, Kenya]]","[[-1, Nairobi, Kenya]]","[[Kawangware, Nairobi, Kenya]]","[[Makueni, Wote, Kenya]]","[[-1, -1, Kenya]]","[[Kawangware, Nairobi, Kenya]]","[[Ka2Angware, Nairobi, Kenya]]","[[-1, Nairobi, Kenya]]","[[Embakasi, Nairobi, Kenya]]","[[-1, Nairobi, Kenya]]","[[-1, Nairobi, Kenya]]"


### 1.1.c. Generate Multilocations through a Composite Split and Bind

For two columns, the locations are stored within delimited string but do not include nearest city or country information. Here the relevant location will be extracted by splitting and the supporting information will be added by binding.

##### Keys for the composite split and bind

In [17]:
# dictionary of keys for the composite split and bind
d_comps = { 
    '231': ['231 Neighbourhoods Lived In', 0, '228 Identify the City', '228b Identify the Country'],
    '238': ['238 Places Returned To ', 0, '228 Identify the City', '228b Identify the Country']
}

# dataframe of keys for the composite split and bind
df_comps = pd.DataFrame(d_comps, index=['c1', 'i', 'c2', 'c3'])
df_comps

,231,238
c1,231 Neighbourhoods Lived In,238 Places Returned To
i,0,0
c2,228 Identify the City,228 Identify the City
c3,228b Identify the Country,228b Identify the Country


##### Loop to perform the composite split and bind

In [18]:
# 01 | Create a new dataframe to store the multilocations
mls_C = pd.DataFrame(index   = survey.index,
                     columns = df_comps.columns)


# 02 | For each composite split and bind to be performed...
for newcol in list(df_comps.columns):
    
    # 02-a | Get each of the keys
    c1 = str( df_comps.loc['c1', newcol] )
    i  = int( df_comps.loc['i', newcol] )
    c2 = str( df_comps.loc['c2', newcol] )
    c3 = str( df_comps.loc['c3', newcol] )
    
    # 02-b | Reset the column and its datatype
    mls_C.at[:, newcol] = None
    
    # 02-c | Gather the individual elements into temporary working columns
    wcl1 = str(newcol+'_l1')
    wcl2 = str(newcol+'_l2')
    wcl3 = str(newcol+'_l3')
    
    mls_C[wcl1] = survey.apply(lambda row : [item[i] for item in delimit(row[c1])], axis=1)
    mls_C[wcl2] = row[c2] # survey.apply(lambda row : row[c2], axis=1)
    mls_C[wcl3] = row[c3] # survey.apply(lambda row : row[c3], axis=1)
    
    # 02-d | Bind the elements
    mls_C[newcol] = mls_C.apply(lambda row : [[l1, 
                                               row[wcl2], 
                                               row[wcl3]] for l1 in row[wcl1]], axis=1)

# Drop the working columns
mls_C = mls_C[list(df_comps.columns)]
    
# Diagnostics
print('\nRows, Columns: {}'.format(mls_C.shape))
print('\nIndex matches original:')
print(pd.Series(mls_C.index == survey.index).value_counts())

# Preview
mls_C.head(10)


Rows, Columns: (1644, 2)

Index matches original:
True    1644
dtype: int64


,231,238
ID,,
276785,"[[Kawangware, Nairobi, Kenya]]","[[-1, Nairobi, Kenya]]"
276788,"[[-1, -1, -1]]","[[-1, -1, -1]]"
276802,"[[Roysambu, Nairobi, Kenya]]","[[-1, Nairobi, Kenya]]"
276814,"[[Umoja, Nairobi, Kenya], [Tena, Nairobi, Kenya]]","[[Nakuru, Nairobi, Kenya], [Kisumu, Nairobi, K..."
276822,"[[Buruburu, Nairobi, Kenya]]","[[-1, Nairobi, Kenya]]"
276931,"[[Kayole, Nairobi, Kenya]]","[[-3, Nairobi, Kenya]]"
276932,"[[Kawangware, Nairobi, Kenya]]","[[-1, Nairobi, Kenya]]"
276934,"[[-1, -1, -1]]","[[-1, -1, -1]]"
276935,"[[Embakasi, Nairobi, Kenya]]","[[-1, Nairobi, Kenya]]"


### 1.1.d. Merge the multilocations into a single dataframe

After all multilocations are generated, they can be reconciled into a single comprehensive dataframe.

##### Concatenation

In [19]:
# shapes of individually computed parts - columns should add up to the concatenated dataframe
print('mls_A: {}'.format(mls_A.shape))
print('mls_B: {}'.format(mls_B.shape))
print('mls_C: {}'.format(mls_C.shape))

# concatenate the columns based on index
mls = pd.concat([mls_A, mls_B, mls_C],
                axis = 'columns',
                join = 'outer',
                ignore_index = False)

# Diagnostics
print('\nmls: {}'.format(mls.shape))
print('\nIndex matches original:')
print(pd.Series(mls.index == survey.index).value_counts())

# Preview
mls.head(10)

mls_A: (1644, 31)
mls_B: (1644, 11)
mls_C: (1644, 2)

mls: (1644, 44)

Index matches original:
True    1644
dtype: int64


,109,127,134,142,152,201,202,206,211,214b,...,100b,101,302,308,229w,229x,229y,229z,231,238
ID,,,,,,,,,,,,,,,,,,,,,
276785,[],[],"[[Kawango, Kisumu, Kenya], [Kawango, Kisumu, K...",[],[],"[[Kawango, Kisumu, Kenya]]","[[Manyatta, Kisumu, Kenya], [Kawangware, Nairo...",[],[],[],...,"[[-1, Kawango, Kenya]]","[[Kisumu, Kisumu, Kenya]]","[[-3, Nairobi, Kenya]]","[[-3, Nairobi, Kenya]]","[[-1, Nairobi, Kenya]]","[[Kawangware, Nairobi, Kenya]]","[[-1, Nairobi, Kenya]]","[[-1, Nairobi, Kenya]]","[[Kawangware, Nairobi, Kenya]]","[[-1, Nairobi, Kenya]]"
276788,[],"[[N/A, N/A, N/A]]","[[N/A, N/A, N/A]]",[],[],[],[],[],[],[],...,"[[-1, Kokal, Kenya]]","[[Oyugis, Oyugis, Kenya]]","[[-1, -1, -1]]","[[-1, -1, -1]]","[[-1, -1, -1]]","[[-1, -1, -1]]","[[-1, -1, -1]]","[[-1, -1, -1]]","[[-1, -1, -1]]","[[-1, -1, -1]]"
276802,[],[],"[[Awendo, Migori, Kenya]]",[],[],"[[Machakos, Machakos, Kenya], [Utawala, Nairob...",[],[],[],[],...,"[[-1, Oyugjs, Kenya]]","[[Oyugis, Oyugis, Kenya]]","[[Utawala, Nairobi, Kenya]]","[[Nairobi Cbd, Nairobi, Kenya]]","[[-1, Nairobi, Kenya]]","[[Roysambu, Nairobi, Kenya]]","[[-1, Nairobi, Kenya]]","[[-1, Nairobi, Kenya]]","[[Roysambu, Nairobi, Kenya]]","[[-1, Nairobi, Kenya]]"
276814,[],"[[Nairobi, Nairobi, Kenya], [London, Nakuru, K...","[[Bondo, Siaya, Kenya]]","[[Kisumu, Kisumu, Kenya]]",[],"[[Wathorego, Kisumu, Kenya]]","[[London, Nakuru, Kenya]]","[[Lolgoria, Narok, Kenya]]","[[Londiani, Kericho, Kenya]]",[],...,"[[-1, Wathorego, Kenya]]","[[Kisumu, Kisumu, Kenya]]","[[Soweto, Nairobi, Kenya]]","[[-3, Nairobi, Kenya]]","[[-1, Nairobi, Kenya]]","[[Kayole, Nairobi, Kenya]]","[[-1, Nairobi, Kenya]]","[[-1, Nairobi, Kenya]]","[[Umoja, Nairobi, Kenya], [Tena, Nairobi, Kenya]]","[[Nakuru, Nairobi, Kenya], [Kisumu, Nairobi, K..."
276822,[],[],"[[None, None, None]]",[],[],"[[Kawango, Maseno, Kenya]]","[[Kedowa, Kericho, Kenya]]",[],[],[],...,"[[-3, Kisumu, Kenya]]","[[-1, -1, Kenya]]","[[Soweto, Nairobi, Kenya]]","[[Kayole, Nairobi, Kenya]]","[[-1, Nairobi, Kenya]]","[[Buru Buru, Nairobi, Kenya]]","[[-1, Nairobi, Kenya]]","[[-1, Nairobi, Kenya]]","[[Buruburu, Nairobi, Kenya]]","[[-1, Nairobi, Kenya]]"
276931,[],"[[London, Nakuru, Kenya]]","[[Usenge, Siaya, Kenya]]","[[Mombasa, Mombasa, Kenya]]","[[Mombasa, Mombasa, Kenya]]","[[Mamboleo, Kisumu, Kenya]]",[],[],[],[],...,"[[-1, Kajulu, Kenya]]","[[Kisumu, Kisumu, Kenya]]","[[Mama Oliech, Nairobi, Kenya]]","[[Stage, Nairobi, Kenya]]","[[-1, Nairobi, Kenya]]","[[Kayole, Nairobi, Kenya]]","[[-1, Nairobi, Kenya]]","[[-1, Nairobi, Kenya]]","[[Kayole, Nairobi, Kenya]]","[[-3, Nairobi, Kenya]]"
276932,[],"[[Kawangware, Nairobi, Kenya]]","[[Na, Na, Na]]",[],[],"[[Kajulu, Kisumu, Kenya]]",[],[],[],[],...,"[[-1, Kajulu, Kenya]]","[[Kisumu, Kisumu, Kenya]]","[[Bus Park, Nairobi, Kenya]]","[[Kawangware, Nairobi, Kenya]]","[[-1, Nairobi, Kenya]]","[[Kawangware, Nairobi, Kenya]]","[[-1, Nairobi, Kenya]]","[[-1, Nairobi, Kenya]]","[[Kawangware, Nairobi, Kenya]]","[[-1, Nairobi, Kenya]]"
276934,[],[],"[[Nairobi, Nairobi, Kenya]]",[],[],"[[Pumwani, Nairobi, Kenya]]","[[Kericho, Kericho, Kenya], [Kawangware, Nairo...",[],[],[],...,"[[Pumwani, Nairobi, Kenya]]","[[-1, -1, Kenya]]","[[Settilite, -1, -1]]","[[Kawangware, -1, -1]]","[[-1, -1, -1]]","[[-1, -1, -1]]","[[-1, -1, -1]]","[[-1, -1, -1]]","[[-1, -1, -1]]","[[-1, -1, -1]]"
276935,[],"[[Nairobi, Nairobi, Kenya]]","[[Wote, Wote, Kenya]]",[],[],"[[Wote, Wote, Kenya]]","[[Kawangware, Nairobi, Kenya]]",[],[],[],...,"[[Makueni, Wote, Kenya]]","[[-1, -1, Kenya]]","[[Kawangware, Nairobi, Kenya]]","[[Ka2Angware, Nairobi, Kenya]]","[[-1, Nairobi, Kenya]]","[[Embakasi, Nairobi, Kenya]]","[[-1, Nairobi, Kenya]]","[[-1, Nairobi, Kenya]]","[[Embakasi, Nairobi, Kenya]]","[[-1, Nairobi, Kenya]]"


## 1.2 Prepare the Multilocations Data for Export

##### Important note on exporting the output multilocations:

During export to a CSV, the elements of the dataframe will be stored and read as string values rather than their literal object datatypes. When importing the exported CSV into another project, python must be prompted to read the Python literal from the string through functions such as __ast.literal_eval()__ .

Though the numpy arrays allow ease of use with several functions from numpy and pandas, if the numpy arrays exported as a CSV and read literally, there will be _problems with reading and constructing the resulting pandas dataframe_ properly due to the conflicts with the numpy arrays with the the similar architecture of the pandas dataframe they are embedded in.

Instead of arrays, the multilocations can be stores as a __tuples__ . Alternatively, they can also can be converted into __delimited string__ similar to the raw data, and use the ml_split() function to be converted into arrays after importing the CSV. The final 

### 1.2.1 Export the multilocations as tuples

##### Function to convert a 2-D array to a nested tuple

In [20]:
def a_to_tuple(a):
    """
    A function that converts a two dimensional array into nested tuples
    
    RETURNS: Multilocation tuples in a tuple.
    """
    # 01 | filter out any empty arrays or NoneTypes from tuple conversion
    # * important to evaluate the tuples literally from the csv without errors
    if a is not None:
        if (len(a) > 0):
            try:
                # 02 | tuple all the items in the array, then tuple the array
                t = tuple([tuple(ml) for ml in a])

                # 03 | ensure the element of tuples have '' before export
                # * important to evaluate the tuples literally from the csv without errors
                t = str(t) 
            except:
                print(a)
                t = None

        else:
            # return empty arrays as null values
            t = None
    else:
        # return nonetypes as null values
        t = None
        
    return t

##### Map the conversion function over every element of the dataframe

In [21]:
# 01 | Map the function
mls_t = mls.applymap(lambda x: a_to_tuple(x))

# 02 | Export to CSV
mls_t.to_csv('data_gen/survey_multilocations/survey_multilocations_tuples.csv')

# Preview
mls_t

,109,127,134,142,152,201,202,206,211,214b,...,100b,101,302,308,229w,229x,229y,229z,231,238
ID,,,,,,,,,,,,,,,,,,,,,
276785,None,None,"(('Kawango', 'Kisumu', 'Kenya'), ('Kawango', '...",None,None,"(('Kawango', 'Kisumu', 'Kenya'),)","(('Manyatta', 'Kisumu', 'Kenya'), ('Kawangware...",None,None,None,...,"(('-1', 'Kawango', 'Kenya'),)","(('Kisumu', 'Kisumu', 'Kenya'),)","(('-3', 'Nairobi', 'Kenya'),)","(('-3', 'Nairobi', 'Kenya'),)","(('-1', 'Nairobi', 'Kenya'),)","(('Kawangware', 'Nairobi', 'Kenya'),)","(('-1', 'Nairobi', 'Kenya'),)","(('-1', 'Nairobi', 'Kenya'),)","(('Kawangware', 'Nairobi', 'Kenya'),)","(('-1', 'Nairobi', 'Kenya'),)"
276788,None,"(('N/A', 'N/A', 'N/A'),)","(('N/A', 'N/A', 'N/A'),)",None,None,None,None,None,None,None,...,"(('-1', 'Kokal', 'Kenya'),)","(('Oyugis', 'Oyugis', 'Kenya'),)","(('-1', '-1', '-1'),)","(('-1', '-1', '-1'),)","(('-1', '-1', '-1'),)","(('-1', '-1', '-1'),)","(('-1', '-1', '-1'),)","(('-1', '-1', '-1'),)","(('-1', '-1', '-1'),)","(('-1', '-1', '-1'),)"
276802,None,None,"(('Awendo', 'Migori', 'Kenya'),)",None,None,"(('Machakos', 'Machakos', 'Kenya'), ('Utawala'...",None,None,None,None,...,"(('-1', 'Oyugjs', 'Kenya'),)","(('Oyugis', 'Oyugis', 'Kenya'),)","(('Utawala', 'Nairobi', 'Kenya'),)","(('Nairobi Cbd', 'Nairobi', 'Kenya'),)","(('-1', 'Nairobi', 'Kenya'),)","(('Roysambu', 'Nairobi', 'Kenya'),)","(('-1', 'Nairobi', 'Kenya'),)","(('-1', 'Nairobi', 'Kenya'),)","(('Roysambu', 'Nairobi', 'Kenya'),)","(('-1', 'Nairobi', 'Kenya'),)"
276814,None,"(('Nairobi', 'Nairobi', 'Kenya'), ('London', '...","(('Bondo', 'Siaya', 'Kenya'),)","(('Kisumu', 'Kisumu', 'Kenya'),)",None,"(('Wathorego', 'Kisumu', 'Kenya'),)","(('London', 'Nakuru', 'Kenya'),)","(('Lolgoria', 'Narok', 'Kenya'),)","(('Londiani', 'Kericho', 'Kenya'),)",None,...,"(('-1', 'Wathorego', 'Kenya'),)","(('Kisumu', 'Kisumu', 'Kenya'),)","(('Soweto', 'Nairobi', 'Kenya'),)","(('-3', 'Nairobi', 'Kenya'),)","(('-1', 'Nairobi', 'Kenya'),)","(('Kayole', 'Nairobi', 'Kenya'),)","(('-1', 'Nairobi', 'Kenya'),)","(('-1', 'Nairobi', 'Kenya'),)","(('Umoja', 'Nairobi', 'Kenya'), ('Tena', 'Nair...","(('Nakuru', 'Nairobi', 'Kenya'), ('Kisumu', 'N..."
276822,None,None,"(('None', 'None', 'None'),)",None,None,"(('Kawango', 'Maseno', 'Kenya'),)","(('Kedowa', 'Kericho', 'Kenya'),)",None,None,None,...,"(('-3', 'Kisumu', 'Kenya'),)","(('-1', '-1', 'Kenya'),)","(('Soweto', 'Nairobi', 'Kenya'),)","(('Kayole', 'Nairobi', 'Kenya'),)","(('-1', 'Nairobi', 'Kenya'),)","(('Buru Buru', 'Nairobi', 'Kenya'),)","(('-1', 'Nairobi', 'Kenya'),)","(('-1', 'Nairobi', 'Kenya'),)","(('Buruburu', 'Nairobi', 'Kenya'),)","(('-1', 'Nairobi', 'Kenya'),)"
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
297661,None,None,"(('Khongoloti', 'Khongoloti', 'Zimbabwe'), ('K...","(('Khongoloti', 'Khongoloti', 'Khongoloti'), (...",None,"(('Berwa', 'Johannesburg', 'South Africa'),)",None,None,None,None,...,"(('Johannesburg', 'Berea', 'South Africa'),)","(('-1', '-1', 'South Africa'),)","(('Born In This City', '-1', '-1'),)","(('University Student', '-1', '-1'),)","(('-1', '-1', '-1'),)","(('-1', '-1', '-1'),)","(('-1', '-1', '-1'),)","(('-1', '-1', '-1'),)","(('-1', '-1', '-1'),)","(('-1', '-1', '-1'),)"
297663,None,"(('Sesambo', 'Lira City', 'Uganda'), ('Sesambo...","(('Khongoloti', 'Khongoloti', 'Zimbabwe'), ('S...",None,None,"(('Sesambo', 'Sesambo', 'Uganda'),)",None,None,None,None,...,"(('-1', 'Sesambo', 'Uganda'),)","(('Lira City', 'Lira City', 'Uganda'),)","(('Was Not Working', 'Johannesburg', 'South Af...","(('Berea', 'Johannesburg', 'South Africa'),)","(('Other (specify)', 'Johannesburg', 'South Af...","(('-1', 'Johannesburg', 'South Africa'),)","(('-1', 'Johannesburg', 'South Africa'),)","(('Berea', 'Johannesburg', 'South Africa'),)","(('Uganda', 'Johannesburg', 'South Africa'), (...","(('-1', 'Johannesburg', 'South Africa'),)"
297666,None,"(('0', '0', '0'),)","(('Lububamshi', 'Katsanga', 'Democratic Republ...","(('Katsanga', 'Katsanga', 'Democratic Repub

### 1.2.2 Export the multilocations as strings with delimiters

 ##### Function to convert a 2-D array to a delimited string

In [22]:
def delimit_a(a, char):
    """
    A supporting function that converts an array into a string with ~ and ^ delimiters.
    
    RETURNS: A delimited string.
    """
    # 01 | Start with an empty string
    string = ''
    
    # 02 | For each item in the array
    for i, item in enumerate(a):
        
        # 02a | Add the string element
        string = string + str(item)
        
        # 02b | Add the delimited except after the last item
        if i < (len(a) - 1):
            string = string + str(char)
        else:
            pass
        
    return str(string)

def a_to_str(a):
    """
    A function that converts a two dimensional array into a delimited string.
    
    RETURNS: Multlocation entries as a string.
    """
    # 01 | filter out any empty arrays or NoneTypes from string conversion
    if a is not None:
        if (len(a) > 0):
            try:
                sub = [delimit_a(ml, '^') for ml in a]
                s = delimit_a(sub, '~')
            except:
                print(a)
                s = None

        else:
            # return empty arrays as null values
            s = None
    else:
        # return nonetypes as null values
        s = None
    
    return s

In [23]:
# 01 | Map the function
mls_s = mls.applymap(lambda x: a_to_str(x))

# 02 | Export to CSV
mls_s.to_csv('data_gen/survey_multilocations/survey_multilocations_strings.csv')

# Preview
mls_s

,109,127,134,142,152,201,202,206,211,214b,...,100b,101,302,308,229w,229x,229y,229z,231,238
ID,,,,,,,,,,,,,,,,,,,,,
276785,None,None,Kawango^Kisumu^Kenya~Kawango^Kisumu^Kenya~Kara...,None,None,Kawango^Kisumu^Kenya,Manyatta^Kisumu^Kenya~Kawangware^Nairobi^Kenya,None,None,None,...,-1^Kawango^Kenya,Kisumu^Kisumu^Kenya,-3^Nairobi^Kenya,-3^Nairobi^Kenya,-1^Nairobi^Kenya,Kawangware^Nairobi^Kenya,-1^Nairobi^Kenya,-1^Nairobi^Kenya,Kawangware^Nairobi^Kenya,-1^Nairobi^Kenya
276788,None,N/A^N/A^N/A,N/A^N/A^N/A,None,None,None,None,None,None,None,...,-1^Kokal^Kenya,Oyugis^Oyugis^Kenya,-1^-1^-1,-1^-1^-1,-1^-1^-1,-1^-1^-1,-1^-1^-1,-1^-1^-1,-1^-1^-1,-1^-1^-1
276802,None,None,Awendo^Migori^Kenya,None,None,Machakos^Machakos^Kenya~Utawala^Nairobi^Kenya,None,None,None,None,...,-1^Oyugjs^Kenya,Oyugis^Oyugis^Kenya,Utawala^Nairobi^Kenya,Nairobi Cbd^Nairobi^Kenya,-1^Nairobi^Kenya,Roysambu^Nairobi^Kenya,-1^Nairobi^Kenya,-1^Nairobi^Kenya,Roysambu^Nairobi^Kenya,-1^Nairobi^Kenya
276814,None,Nairobi^Nairobi^Kenya~London^Nakuru^Kenya,Bondo^Siaya^Kenya,Kisumu^Kisumu^Kenya,None,Wathorego^Kisumu^Kenya,London^Nakuru^Kenya,Lolgoria^Narok^Kenya,Londiani^Kericho^Kenya,None,...,-1^Wathorego^Kenya,Kisumu^Kisumu^Kenya,Soweto^Nairobi^Kenya,-3^Nairobi^Kenya,-1^Nairobi^Kenya,Kayole^Nairobi^Kenya,-1^Nairobi^Kenya,-1^Nairobi^Kenya,Umoja^Nairobi^Kenya~Tena^Nairobi^Kenya,Nakuru^Nairobi^Kenya~Kisumu^Nairobi^Kenya
276822,None,None,None^None^None,None,None,Kawango^Maseno^Kenya,Kedowa^Kericho^Kenya,None,None,None,...,-3^Kisumu^Kenya,-1^-1^Kenya,Soweto^Nairobi^Kenya,Kayole^Nairobi^Kenya,-1^Nairobi^Kenya,Buru Buru^Nairobi^Kenya,-1^Nairobi^Kenya,-1^Nairobi^Kenya,Buruburu^Nairobi^Kenya,-1^Nairobi^Kenya
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
297661,None,None,Khongoloti^Khongoloti^Zimbabwe~Khongoloti^Khon...,Khongoloti^Khongoloti^Khongoloti~Khongoloti^Kh...,None,Berwa^Johannesburg^South Africa,None,None,None,None,...,Johannesburg^Berea^South Africa,-1^-1^South Africa,Born In This City^-1^-1,University Student^-1^-1,-1^-1^-1,-1^-1^-1,-1^-1^-1,-1^-1^-1,-1^-1^-1,-1^-1^-1
297663,None,Sesambo^Lira City^Uganda~Sesambo^Liracity^Ugan...,Khongoloti^Khongoloti^Zimbabwe~Sesambo^Sesambo...,None,None,Sesambo^Sesambo^Uganda,None,None,None,None,...,-1^Sesambo^Uganda,Lira City^Lira City^Uganda,Was Not Working^Johannesburg^South Africa,Berea^Johannesburg^South Africa,Other (specify)^Johannesburg^South Africa,-1^Johannesburg^South Africa,-1^Johannesburg^South Africa,Berea^Johannesburg^South Africa,Uganda^Johannesburg^South Africa~Berea^Johanne...,-1^Johannesburg^South Africa
297666,None,0^0^0,Lububamshi^Katsanga^Democratic Republic Of The...,Katsanga^Katsanga^Democratic Republic Of The C...,None,Lububamshi^Katsanga^Democratic Republic Of The...,Bloemfontein^Bloemfontein^South Africa,Yeoville^Johannesburg^South Africa,Berea^Johannesburg^South Africa,None,...,-1^Lububamshi^Democratic Republic Of The Congo,Katsanga^Katsanga^Democratic Republic Of The C...,Berea^Johannesburg^South Africa,Johannesburg Berea^Johannesburg^South Africa,Joburg CBD^Johannesburg^South Africa,-1^Johannesburg^South Africa,-1^Johannesburg^South Africa,-1^Johannesburg^South Africa,Yeoville^Johannesburg^South Africa~Berea^Johan...,Yeoville^Johannesburg^South Africa~Bloemfontei...


## 1.3 Reloading Multilocations Output after Export

The final section of this notebook provides examples to restore the data structure of the multilocations after exporting the multilocations to a CSV file.

### 1.3.1. Reading tuples from an exported CSV file

##### Libraries

In [24]:
# package dependency for literal_eval()
import ast

##### Function to evaluate the strings of the CSV as tuple literals

In [25]:
# function to map over the string elements of the dataframe
def str_eval(e):
    """
    A function to evaluate string element in a dataframe literally, such as the multilocation tuples.
    
    RETURNS: The Python literal. 
    """
    # 01 | Ensure the element is read by ast.literal_eval() as a string to avoid errors.
    e = str(e)
    
    # 02 | Filter out any null values
    if (e != 'nan'):
        
        # 03 | Evaluate the string literally
        try:
            ml = ast.literal_eval(e)
        
        # If unable to read the string, print it so it can be traced back to address issues
        except:
            print(e)
            ml = None
    
    # Return null values as null values
    else:
        ml = None
        
    return ml

##### Map the function over the imported CSV

In [26]:
# 01 | Load csv and set the index
t_test = pd.read_csv('data_gen/survey_multilocations/survey_multilocations_tuples.csv')
t_test = t_test.set_index('ID')

# Preview
print('\nBefore: ')
display(t_test.head(3))

# 02 | Map the function
t_test = t_test.applymap(lambda x: str_eval(str(x)))

# Preview
print('\nAfter: ')
display(t_test.head(3))


Before: 


,109,127,134,142,152,201,202,206,211,214b,...,100b,101,302,308,229w,229x,229y,229z,231,238
ID,,,,,,,,,,,,,,,,,,,,,
276785,NaN,NaN,"(('Kawango', 'Kisumu', 'Kenya'), ('Kawango', '...",NaN,NaN,"(('Kawango', 'Kisumu', 'Kenya'),)","(('Manyatta', 'Kisumu', 'Kenya'), ('Kawangware...",NaN,NaN,NaN,...,"(('-1', 'Kawango', 'Kenya'),)","(('Kisumu', 'Kisumu', 'Kenya'),)","(('-3', 'Nairobi', 'Kenya'),)","(('-3', 'Nairobi', 'Kenya'),)","(('-1', 'Nairobi', 'Kenya'),)","(('Kawangware', 'Nairobi', 'Kenya'),)","(('-1', 'Nairobi', 'Kenya'),)","(('-1', 'Nairobi', 'Kenya'),)","(('Kawangware', 'Nairobi', 'Kenya'),)","(('-1', 'Nairobi', 'Kenya'),)"
276788,NaN,"(('N/A', 'N/A', 'N/A'),)","(('N/A', 'N/A', 'N/A'),)",NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,"(('-1', 'Kokal', 'Kenya'),)","(('Oyugis', 'Oyugis', 'Kenya'),)","(('-1', '-1', '-1'),)","(('-1', '-1', '-1'),)","(('-1', '-1', '-1'),)","(('-1', '-1', '-1'),)","(('-1', '-1', '-1'),)","(('-1', '-1', '-1'),)","(('-1', '-1', '-1'),)","(('-1', '-1', '-1'),)"
276802,NaN,NaN,"(('Awendo', 'Migori', 'Kenya'),)",NaN,NaN,"(('Machakos', 'Machakos', 'Kenya'), ('Utawala'...",NaN,NaN,NaN,NaN,...,"(('-1', 'Oyugjs', 'Kenya'),)","(('Oyugis', 'Oyugis', 'Kenya'),)","(('Utawala', 'Nairobi', 'Kenya'),)","(('Nairobi Cbd', 'Nairobi', 'Kenya'),)","(('-1', 'Nairobi', 'Kenya'),)","(('Roysambu', 'Nairobi', 'Kenya'),)","(('-1', 'Nairobi', 'Kenya'),)","(('-1', 'Nairobi', 'Kenya'),)","(('Roysambu', 'Nairobi', 'Kenya'),)","(('-1', 'Nairobi', 'Kenya'),)"



After: 


,109,127,134,142,152,201,202,206,211,214b,...,100b,101,302,308,229w,229x,229y,229z,231,238
ID,,,,,,,,,,,,,,,,,,,,,
276785,None,None,"((Kawango, Kisumu, Kenya), (Kawango, Kisumu, K...",None,None,"((Kawango, Kisumu, Kenya),)","((Manyatta, Kisumu, Kenya), (Kawangware, Nairo...",None,None,None,...,"((-1, Kawango, Kenya),)","((Kisumu, Kisumu, Kenya),)","((-3, Nairobi, Kenya),)","((-3, Nairobi, Kenya),)","((-1, Nairobi, Kenya),)","((Kawangware, Nairobi, Kenya),)","((-1, Nairobi, Kenya),)","((-1, Nairobi, Kenya),)","((Kawangware, Nairobi, Kenya),)","((-1, Nairobi, Kenya),)"
276788,None,"((N/A, N/A, N/A),)","((N/A, N/A, N/A),)",None,None,None,None,None,None,None,...,"((-1, Kokal, Kenya),)","((Oyugis, Oyugis, Kenya),)","((-1, -1, -1),)","((-1, -1, -1),)","((-1, -1, -1),)","((-1, -1, -1),)","((-1, -1, -1),)","((-1, -1, -1),)","((-1, -1, -1),)","((-1, -1, -1),)"
276802,None,None,"((Awendo, Migori, Kenya),)",None,None,"((Machakos, Machakos, Kenya), (Utawala, Nairob...",None,None,None,None,...,"((-1, Oyugjs, Kenya),)","((Oyugis, Oyugis, Kenya),)","((Utawala, Nairobi, Kenya),)","((Nairobi Cbd, Nairobi, Kenya),)","((-1, Nairobi, Kenya),)","((Roysambu, Nairobi, Kenya),)","((-1, Nairobi, Kenya),)","((-1, Nairobi, Kenya),)","((Roysambu, Nairobi, Kenya),)","((-1, Nairobi, Kenya),)"


### 1.3.2. Reading delimited string from an exported CSV file

##### Function to split the delimited strings in the CSV file

In [27]:
# function to split
def delimit(string):
    """
    A function that splits a string by its delimiters
    
    RETURNS: Separated strings in an array
    """
    # 01 | split the string into discrete entries using delimiter '~'
    subs = str(string).split('~')
    
    # 02 | split the entry string into discrete component using delimiter '^'
    a = [str(item).split('^') for item in subs]
    
    return a

# function to filter and split
def str_split(e):
    """
    A function that splits a string by its delimiters
    
    RETURNS: Separated strings in an array
    """
    # 01 | Ensure the element is read by ast.literal_eval() as a string to avoid errors.
    e = str(e)
    
    # 02 | Filter out any null values
    if (e != 'nan'):
        a = delimit(e)
    
    else:
        a = None
    
    return a

##### Map the function over the imported CSV

In [28]:
# 01 | Load csv and set the index
s_test = pd.read_csv('data_gen/survey_multilocations/survey_multilocations_strings.csv')
s_test = s_test.set_index('ID')

# Preview
print('\nBefore: ')
display(s_test.head(3))

# 02 | Map the split function
s_test = s_test.applymap(lambda x: str_split(x))

# Preview
print('\nAfter (Arrays): ')
display(s_test.head(3))

# 03 | Map the tuples function
s_test = s_test.applymap(lambda x: a_to_tuple(x))

# Preview
print('\nAfter (Tuples): ')
display(s_test.head(3))


Before: 


,109,127,134,142,152,201,202,206,211,214b,...,100b,101,302,308,229w,229x,229y,229z,231,238
ID,,,,,,,,,,,,,,,,,,,,,
276785,NaN,NaN,Kawango^Kisumu^Kenya~Kawango^Kisumu^Kenya~Kara...,NaN,NaN,Kawango^Kisumu^Kenya,Manyatta^Kisumu^Kenya~Kawangware^Nairobi^Kenya,NaN,NaN,NaN,...,-1^Kawango^Kenya,Kisumu^Kisumu^Kenya,-3^Nairobi^Kenya,-3^Nairobi^Kenya,-1^Nairobi^Kenya,Kawangware^Nairobi^Kenya,-1^Nairobi^Kenya,-1^Nairobi^Kenya,Kawangware^Nairobi^Kenya,-1^Nairobi^Kenya
276788,NaN,N/A^N/A^N/A,N/A^N/A^N/A,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,-1^Kokal^Kenya,Oyugis^Oyugis^Kenya,-1^-1^-1,-1^-1^-1,-1^-1^-1,-1^-1^-1,-1^-1^-1,-1^-1^-1,-1^-1^-1,-1^-1^-1
276802,NaN,NaN,Awendo^Migori^Kenya,NaN,NaN,Machakos^Machakos^Kenya~Utawala^Nairobi^Kenya,NaN,NaN,NaN,NaN,...,-1^Oyugjs^Kenya,Oyugis^Oyugis^Kenya,Utawala^Nairobi^Kenya,Nairobi Cbd^Nairobi^Kenya,-1^Nairobi^Kenya,Roysambu^Nairobi^Kenya,-1^Nairobi^Kenya,-1^Nairobi^Kenya,Roysambu^Nairobi^Kenya,-1^Nairobi^Kenya



After (Arrays): 


,109,127,134,142,152,201,202,206,211,214b,...,100b,101,302,308,229w,229x,229y,229z,231,238
ID,,,,,,,,,,,,,,,,,,,,,
276785,None,None,"[[Kawango, Kisumu, Kenya], [Kawango, Kisumu, K...",None,None,"[[Kawango, Kisumu, Kenya]]","[[Manyatta, Kisumu, Kenya], [Kawangware, Nairo...",None,None,None,...,"[[-1, Kawango, Kenya]]","[[Kisumu, Kisumu, Kenya]]","[[-3, Nairobi, Kenya]]","[[-3, Nairobi, Kenya]]","[[-1, Nairobi, Kenya]]","[[Kawangware, Nairobi, Kenya]]","[[-1, Nairobi, Kenya]]","[[-1, Nairobi, Kenya]]","[[Kawangware, Nairobi, Kenya]]","[[-1, Nairobi, Kenya]]"
276788,None,"[[N/A, N/A, N/A]]","[[N/A, N/A, N/A]]",None,None,None,None,None,None,None,...,"[[-1, Kokal, Kenya]]","[[Oyugis, Oyugis, Kenya]]","[[-1, -1, -1]]","[[-1, -1, -1]]","[[-1, -1, -1]]","[[-1, -1, -1]]","[[-1, -1, -1]]","[[-1, -1, -1]]","[[-1, -1, -1]]","[[-1, -1, -1]]"
276802,None,None,"[[Awendo, Migori, Kenya]]",None,None,"[[Machakos, Machakos, Kenya], [Utawala, Nairob...",None,None,None,None,...,"[[-1, Oyugjs, Kenya]]","[[Oyugis, Oyugis, Kenya]]","[[Utawala, Nairobi, Kenya]]","[[Nairobi Cbd, Nairobi, Kenya]]","[[-1, Nairobi, Kenya]]","[[Roysambu, Nairobi, Kenya]]","[[-1, Nairobi, Kenya]]","[[-1, Nairobi, Kenya]]","[[Roysambu, Nairobi, Kenya]]","[[-1, Nairobi, Kenya]]"



After (Tuples): 


,109,127,134,142,152,201,202,206,211,214b,...,100b,101,302,308,229w,229x,229y,229z,231,238
ID,,,,,,,,,,,,,,,,,,,,,
276785,None,None,"(('Kawango', 'Kisumu', 'Kenya'), ('Kawango', '...",None,None,"(('Kawango', 'Kisumu', 'Kenya'),)","(('Manyatta', 'Kisumu', 'Kenya'), ('Kawangware...",None,None,None,...,"(('-1', 'Kawango', 'Kenya'),)","(('Kisumu', 'Kisumu', 'Kenya'),)","(('-3', 'Nairobi', 'Kenya'),)","(('-3', 'Nairobi', 'Kenya'),)","(('-1', 'Nairobi', 'Kenya'),)","(('Kawangware', 'Nairobi', 'Kenya'),)","(('-1', 'Nairobi', 'Kenya'),)","(('-1', 'Nairobi', 'Kenya'),)","(('Kawangware', 'Nairobi', 'Kenya'),)","(('-1', 'Nairobi', 'Kenya'),)"
276788,None,"(('N/A', 'N/A', 'N/A'),)","(('N/A', 'N/A', 'N/A'),)",None,None,None,None,None,None,None,...,"(('-1', 'Kokal', 'Kenya'),)","(('Oyugis', 'Oyugis', 'Kenya'),)","(('-1', '-1', '-1'),)","(('-1', '-1', '-1'),)","(('-1', '-1', '-1'),)","(('-1', '-1', '-1'),)","(('-1', '-1', '-1'),)","(('-1', '-1', '-1'),)","(('-1', '-1', '-1'),)","(('-1', '-1', '-1'),)"
276802,None,None,"(('Awendo', 'Migori', 'Kenya'),)",None,None,"(('Machakos', 'Machakos', 'Kenya'), ('Utawala'...",None,None,None,None,...,"(('-1', 'Oyugjs', 'Kenya'),)","(('Oyugis', 'Oyugis', 'Kenya'),)","(('Utawala', 'Nairobi', 'Kenya'),)","(('Nairobi Cbd', 'Nairobi', 'Kenya'),)","(('-1', 'Nairobi', 'Kenya'),)","(('Roysambu', 'Nairobi', 'Kenya'),)","(('-1', 'Nairobi', 'Kenya'),)","(('-1', 'Nairobi', 'Kenya'),)","(('Roysambu', 'Nairobi', 'Kenya'),)","(('-1', 'Nairobi', 'Kenya'),)"
